<a href="https://colab.research.google.com/github/HimanshuSahoo/PythonCode/blob/main/AWS_Lambda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import json
import boto3
import datetime
from datetime import date
import time
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('configuration.properties')

In [ ]:
areaFile=config['MetadataFiles']['areaMetadata']
siteFile=config['MetadataFiles']['sitesMetadata']
bucketName=config['S3Configuration']['bucketName']
s3FilePrefix=config['S3Configuration']['s3FilePrefix']
s3FileSuffix=config['S3Configuration']['s3FileSuffix']
filenameIntials=config['S3Configuration']['filename']
fileExtension=config['S3Configuration']['fileExtension']

In [ ]:
print(areaFile,siteFile,bucketName,s3FilePrefix,filenameIntials,fileExtension)

In [ ]:
def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    

In [ ]:
for record in event['Records']:
        
        #Kinesis data is base64 encoded so decode here
        payload = base64.b64decode(record['kinesis']['data']).decode('utf-8')
        payloadTest=payload
        
        payloadParsed=json.loads(payload)
        areaId1=payloadParsed.get('areaId')
        
        
        notificationType=payloadParsed.get('type')
        
        
        
        
        if (notificationType=='COUNTING'):
            payloadParsed['countIn']=None
            payloadParsed['countOut']=None
            finalPayload=json.dumps(payloadParsed)
        elif (notificationType=='TRAFFIC'):
            payloadParsed['count']=None
            finalPayload=json.dumps(payloadParsed)

In [ ]:
query1="SELECT d.siteId FROM S3Object[*].areas[*] d where d.id='%s'"%(areaId1)

In [ ]:
#below is the code for S3 select query
        s3 = boto3.client('s3')
        resp = s3.select_object_content(
            Bucket=bucketName,
            Key=areaFile,
            ExpressionType='SQL',
            Expression=query1,
            InputSerialization = {'JSON': {"Type": "Lines"}},
            OutputSerialization = {'JSON': {}}
        )
        

In [ ]:
print("response from query1 is : ",resp)

In [ ]:
 count=0
        for event in resp['Payload']:
            if ('Records' in event) and (count==0):
                records = event['Records']['Payload'].decode('utf-8')
            elif 'Stats' in event and (count==0):
                records=None
            count=count+1

In [ ]:
 if (records==None):
            print("putting to unmapped country please update the metadata JSON of pointgrab related to areas")
            country='unmappedCountry'
        elif (records != None):
            parsedRecord=json.loads(records)     
            siteId=parsedRecord.get('siteId')
            
            query2="SELECT d.location.country FROM S3Object[*].sites[*] d where d.id='%s'"%(siteId)
            
            resp1 = s3.select_object_content(
                Bucket=bucketName,
                Key=siteFile,
                ExpressionType='SQL',
                Expression=query2,
                InputSerialization = {'JSON': {"Type": "Lines"}},
                OutputSerialization = {'JSON': {}}
            )

In [ ]:
print("response from query2 is : ",resp1)
            
                
            for event in resp1['Payload']:
                if 'Records' in event:
                    records1 = event['Records']['Payload'].decode('utf-8')
                    #print(records1)    
                    
            parsedrecord1=json.loads(records1)
            countryUpper=parsedrecord1.get("country")
            
            country=countryUpper.lower()


In [ ]:
 today = str(date.today())

In [ ]:
 #s3://gwes-dev-data/sftp/pointgrab_data/testLambda/
        #bucket='sftp/'+'pointgrab/'+country+'/'+'source/'+'sensor/'+today+'/'
        bucket=s3FilePrefix + country +s3FileSuffix +today+'/'
        x= datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        filename=filenameIntials + x
        filepath=bucket+filename+fileExtension
        print("path of S3 file is : ",filepath)
        
        encoded_string = payload.encode("utf-8")

In [ ]:
s3 = boto3.resource("s3")
s3response=s3.Bucket(bucketName).put_object(Key=filepath, Body=finalPayload)
print("response from S3 client is : ",s3response)
    
return "lamba invocation ended"